In [1]:
import openmc
import numpy as np
from openmc.data import AVOGADRO, atomic_mass, atomic_weight

In [2]:
def salt_dens_f(t):
    return 235.0 - (0.02317 * t)

def salt_dens(t):
    return 3.752 - (6.68e-4 * t)

def print_percents(mat, percent_type='ao'):
    if percent_type == 'ao':
        at_percents = []
        nucs = []
        at_mass = []
        for nuc, pt, tp in mat.nuclides:
            nucs.append(nuc)
            at_percents.append(pt)
            at_mass.append(atomic_mass(nuc))

        at_percents = np.array(at_percents)
        at_mass = np.array(at_mass)
        print('Atom Percent:')
        for i, nuc in enumerate(nucs):
            print(f'{nuc}: {100 * at_percents[i]}')

        mass_percents = at_percents*at_mass / np.dot(at_percents, at_mass)
        print('\nMass Percent:')
        for i, nuc in enumerate(nucs):
            print(f'{nuc}: {mass_percents[i] * 100}')
    else:
        mass_percents = []
        nucs = []
        at_mass = []
        for nuc, pt, tp in mat.nuclides:
            nucs.append(nuc)
            mass_percents.append(pt)
            at_mass.append(atomic_mass(nuc))

        mass_percents = np.array(mass_percents)
        at_mass = np.array(at_mass)
        print('Mass Percent:')
        for i, nuc in enumerate(nucs):
            print(f'{nuc}: {100 * mass_percents[i]}')

        at_percents = (mass_percents/at_mass) / np.dot(mass_percents, 1/at_mass)
        print('\nAtom Percent:')
        for i, nuc in enumerate(nucs):
            print(f'{nuc}: {at_percents[i] * 100}')
    
    return nucs, at_percents, mass_percents

In [3]:
salt_dens(900-273.15)

3.3332642

In [4]:
comps = np.array([1, 1, 1, 2, 1, 4, 1, 4])

# Ref values
vals = np.array([71.7,71.7,16.0,16.0, 12.0,12.0, 0.3,0.3])

nucs = (['Li', 'F', 'Be', 'F', 'Th', 'F', 'U', 'F'])
vals = comps*vals
tots = ([vals[0], vals[1] + vals[3] + vals[5] + vals[7], vals[2], vals[4], vals[6]])
tots = tots / np.sum(tots) * 100
nucs = [nucs[0], nucs[1], nucs[2], nucs[4], nucs[6]]

In [5]:
tots

array([28.35112693, 60.45867932,  6.32661131,  4.74495848,  0.11862396])

### Reference Salt

In [6]:
# reference
components = {'Li': {'percent': tots[0]/100, 
                     'enrichment': 95.995, 
                     'enrichment_target': 'Li7',
                     'enrichment_type': 'wo'}, 
              'F19': tots[1]/100, 
              'Be9': tots[2]/100, 
              'Th232': tots[3]/100, 
              'U233': tots[4]/100}

mat = openmc.Material()
mat.add_components(components, percent_type='ao')
mat

Material
	ID             =	1
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Li6            =	0.01315629377894086 [ao]
	Li7            =	0.270354975497453 [ao]
	F19            =	0.6045867931988929 [ao]
	Be9            =	0.06326611308817714 [ao]
	Th232          =	0.047449584816132866 [ao]
	U233           =	0.0011862396204033216 [ao]

In [7]:
nucs, at_percents, mass_percents = print_percents(mat)

Atom Percent:
Li6: 1.315629377894086
Li7: 27.0354975497453
F19: 60.458679319889285
Be9: 6.326611308817714
Th232: 4.744958481613287
U233: 0.11862396204033215

Mass Percent:
Li6: 0.31256052647803345
Li7: 7.491697313173247
F19: 45.36613892305266
Be9: 2.2519421042294803
Th232: 43.48582294966552
U233: 1.091838183401057


In [8]:
# Amounts to put in chapter 4 of thesis
for i, nuc in enumerate(nucs):
    print(f'{nuc}: {100 * np.round(at_percents[i],8)}')

Li6: 1.315629
Li7: 27.035498000000004
F19: 60.458679000000004
Be9: 6.326611
Th232: 4.744958
U233: 0.11862400000000001


In [9]:
np.sum([np.round(at_percents[i], 8) for i in range(0,len(nucs))])

0.9999999900000002

In [10]:
# Amounts to put in chapter 4 of thesis
for i, nuc in enumerate(nucs):
    print(f'{nuc}: {100 * np.round(mass_percents[i],8)}')
    

Li6: 0.312561
Li7: 7.491697
F19: 45.366139
Be9: 2.2519419999999997
Th232: 43.485823
U233: 1.091838


In [11]:
np.sum([np.round(mass_percents[i], 8) for i in range(0,len(nucs))])

1.0

## Model Salt

In [12]:
# Optimized values
vals = np.array([71.75,71.75,16.0,16.0, 12.0,12.0, 0.25,0.25])

vals = comps*vals
tots = ([vals[0], vals[1] + vals[3] + vals[5] + vals[7], vals[2], vals[4], vals[6]])
x0 = tots
tots = tots / np.sum(tots) * 100
tots

array([28.38773492, 60.43521266,  6.33036597,  4.74777448,  0.09891197])

In [13]:
components = {'Li7': tots[0]/100, 
              'F19': tots[1]/100, 
              'Be9': tots[2]/100, 
              'Th232': tots[3]/100, 
              'U233': tots[4]/100}

mat = openmc.Material()
mat.add_components(components, percent_type='ao')
mat

Material
	ID             =	2
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Li7            =	0.2838773491592483 [ao]
	F19            =	0.6043521266073195 [ao]
	Be9            =	0.06330365974282888 [ao]
	Th232          =	0.04747774480712166 [ao]
	U233           =	0.0009891196834817012 [ao]

In [14]:
nucs, at_percents, mass_percents = print_percents(mat)

Atom Percent:
Li7: 28.387734915924828
F19: 60.43521266073195
Be9: 6.330365974282888
Th232: 4.747774480712167
U233: 0.09891196834817012

Mass Percent:
Li7: 7.875052818898849
F19: 45.39835279026769
Be9: 2.2557541472019182
Th232: 43.55943488909532
U233: 0.911405354536209


In [15]:
# Amounts to put in chapter 4 of thesis
for i, nuc in enumerate(nucs):
    print(f'{nuc}: {100 * np.round(at_percents[i],8)}')
    

Li7: 28.387734999999996
F19: 60.435213
Be9: 6.330366
Th232: 4.747774
U233: 0.098912


In [16]:
np.sum([np.round(at_percents[i], 8) for i in range(0,len(nucs))])

0.9999999999999999

In [17]:
# Amounts to put in chapter 4 of thesis
for i, nuc in enumerate(nucs):
    print(f'{nuc}: {100 * np.round(mass_percents[i],8)}')
    

Li7: 7.875053
F19: 45.398353
Be9: 2.255754
Th232: 43.559435
U233: 0.911405


In [18]:
np.sum([np.round(mass_percents[i], 8) for i in range(0,len(nucs))])

1.0

In [19]:
# Values in mats/msbr_saltproc_prepr_comp.ini
components = {'Li7': 7.87474673879085/100, 
              'F19': 45.4003012179284/100, 
              'Be9': 2.25566879138321/100, 
              'Th232': 43.5579130482336/100, 
              'U233': 0.911370203663893/100}

mat = openmc.Material()
mat.add_components(components, percent_type='wo')
mat

Material
	ID             =	3
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Li7            =	0.0787474673879085 [wo]
	F19            =	0.454003012179284 [wo]
	Be9            =	0.0225566879138321 [wo]
	Th232          =	0.435579130482336 [wo]
	U233           =	0.00911370203663893 [wo]

In [20]:
nucs, at_percents, mass_percents = print_percents(mat, percent_type='wo')

Mass Percent:
Li7: 7.87474673879085
F19: 45.4003012179284
Be9: 2.25566879138321
Th232: 43.5579130482336
U233: 0.9113702036638929

Atom Percent:
Li7: 28.386324651090604
F19: 60.43715299180763
Be9: 6.330057997083041
Th232: 4.747557275882828
U233: 0.09890708413590246


In [21]:
# Fixed point iteration to find the values for the actual refs
x1, x2, x3, x4, x5 = at_percents

A = np.array([[x1, x1, x1, x1, x1],
              [x2, x2, x2, x2, x2],
              [x3, x3, x3, x3, x3],
              [x4, x4, x4, x4, x4],
              [x5, x5, x5, x5, x5]])
x = x0
x_new = np.dot(A,x)

i = 1
# Fixed point iteration
while not(np.all(x_new == x)):
    i += 1
    x = x_new
    x_new = np.dot(A, x)

In [22]:
x

array([ 71.74643556, 152.75490419,  15.99922159,  11.99945101,
         0.24998766])

In [23]:
x0

[71.75, 152.75, 16.0, 12.0, 0.25]

It looks like the difference is due to rounding errors/differences in the mass values used.

## Hastelloy N

In [24]:
components = {'Mo': 0.12, 
              'Cr': 0.07,
              'Fe': 0.03,
              'C': 0.0006,
              'Mn': 0.0035,
              'Si': 0.001,
              'W': 0.001,
              'Al': 0.001,
              'Ti': 0.0125, #avg
              'Cu': 0.001,
              'Co': 0.002,
              'P': 0.00015,
              'S': 0.00015,
              'B': 0.000010,
              'Hf': 0.01,
              'Nb': 0.01}

tot = 0
wts = []
for nuc, wt in components.items():
    wts.append(wt*100)
    tot += wt
nickel = 1 - tot

components.update({'Ni': nickel})

In [25]:
elem_mass = []
elem_wt = []
elems = []
for elem, wt in components.items():
    elem_mass.append(atomic_weight(elem))
    elem_wt.append(wt)
    elems.append(elem)
    
elem_mass = np.array(elem_mass)
elem_wt = np.array(elem_wt)

elem_at = (elem_wt / elem_mass) / (np.sum(elem_wt/elem_mass))


for i, elem in enumerate(elems):
    print(f'{elem}: {elem_wt[i] * 100}')

Mo: 12.0
Cr: 7.000000000000001
Fe: 3.0
C: 0.06
Mn: 0.35000000000000003
Si: 0.1
W: 0.1
Al: 0.1
Ti: 1.25
Cu: 0.1
Co: 0.2
P: 0.015
S: 0.015
B: 0.001
Hf: 1.0
Nb: 1.0
Ni: 73.709


In [26]:
# Amounts to put in chapter 4 of thesis
for i, elem in enumerate(elems):
    print(f'{elem}: {np.round(elem_at[i], 5) * 100}')

Mo: 7.64
Cr: 8.224
Fe: 3.282
C: 0.305
Mn: 0.38899999999999996
Si: 0.218
W: 0.033
Al: 0.22599999999999998
Ti: 1.595
Cu: 0.096
Co: 0.207
P: 0.03
S: 0.029
B: 0.006
Hf: 0.34199999999999997
Nb: 0.658
Ni: 76.72


In [27]:
mat = openmc.Material()
mat.add_components(components, percent_type='wo')
mat

Material
	ID             =	4
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Mo100          =	0.012175225961647808 [wo]
	Mo92           =	0.01683826965647346 [wo]
	Mo94           =	0.010789582056756033 [wo]
	Mo95           =	0.018840557266836578 [wo]
	Mo96           =	0.01999840362163272 [wo]
	Mo97           =	0.011613114609036522 [wo]
	Mo98           =	0.02974484682761688 [wo]
	Cr50           =	0.0029215806359028196 [wo]
	Cr52           =	0.058589553645942136 [wo]
	Cr53           =	0.006771510773072629 [wo]
	Cr54           =	0.00171735494508243 [wo]
	Fe54           =	0.0016936674668120565 [wo]
	Fe56           =	0.027570458633868738 [wo]
	Fe57           =	0.0006481105841023909 [wo]
	Fe58           =	8.776331521681765e-05 [wo]
	C0             =	0.0006       [wo]
	Mn55           =	0.0035       [wo]
	Si28           =	0.0009187351728283676 [wo]
	Si29           =	4.831750292786396e-05 [wo]
	Si30           =	3.2947324243768426e-05 [wo